In [59]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [60]:
def households(file,year):
    # Read in file
    df = pd.read_csv(file)

    # Get rid of spaces in Labels
    df['Label'] = df['Label'].apply(lambda x: x.lstrip())

    
    # Get average household and family size and total households
    average_household_size=  float(df.iloc[-2:]['Estimate'][14])
    average_family_size= float(df.iloc[-2:]['Estimate'][15])
    total_households = df.iloc[0]['Estimate']
    total_households = int(total_households.replace(',',''))

    # Convert percentage estimates to float
    df['Percentage'] = df['Estimate'].apply(lambda x: float(x.rstrip('%'))/100 if '%' in x else None)

    # Move Total Household Population from row to column
    df['Total Household Population'] = int(df.iloc[0]['Estimate'].replace(',',''))
    df.dropna(inplace=True)

    # Drop unnecessary columns
    df.drop(columns=['Estimate','Margin of Error'], inplace=True)

    # Calculate population estimates
    df['Population Estimate'] = df['Percentage']*df['Total Household Population']

    # Drop unnecessary columns
    df.drop(columns=['Percentage','Total Household Population'], inplace=True)

    

    # Change index and transpose table
    df.set_index('Label', inplace=True)
    df = df.T

    # Add average household and family size to table. Also add total household count
    
    df.insert(0, 'Average Household Size', average_household_size)
    df.insert(1, 'Average Family Size', average_family_size)
    df.insert(2, 'Total Household Count', total_households)

    #  Rename and drop columns
    try:
        df.drop(columns=['With own children under 18 years','Male householder','Female householder'], inplace=True)
    except:
        df.drop(columns=['With own children of the householder under 18 years','Male householder','Female householder'], inplace=True)
    df.rename(columns={'Family households': 'Multi Generational Family Households', 'Married-couple family': 'Single Generation Married Couple', 'Female householder, no husband present, family': 'Multi Generational Divorced Family Households','Nonfamily households': 'Non Family Households', 'Female householder, no spouse present, family': 'Multi Generational Divorced Family Households'}, inplace=True)
    


    # Add new columns
    df['Non Family Households (Living Alone)'] = df.iloc[:,7] + df.iloc[:,9]
    df['Non Family Households (Not Living Alone)'] = df.iloc[:,8] + df.iloc[:,10]

    # Drop unnecessary columns
    df.drop(columns=['Living alone','Not living alone'], inplace=True)


    # Change index to year
    df.index = [year]

    return df


In [61]:
def poverty_rates(file_poverty_rates,year):
    # Read in file
    df = pd.read_csv(file_poverty_rates)

    # Get rid of spaces in Labels
    df['Label'] = df['Label'].apply(lambda x: x.lstrip())

    # Convert percentage estimates to float
    df['Percentage'] = df['Estimate'].apply(lambda x: float(x.rstrip('%'))/100 if '%' in x else None)

    # Drop unnecessary columns
    df.drop(columns=['Estimate','Margin of Error'], inplace=True)
    

    # Change index and transpose table
    df.set_index('Label', inplace=True)
    df = df.T

    # Change index to year
    df.index = [year]
    try:
        return df[['All families','Married-couple family', 'Female householder, no husband present, family']]
    except:
        return df[['All families','Married-couple family', 'Female householder, no spouse present, family']]



In [66]:
def combine_frames(df_household, df_poverty):
    df_household['Total Families Under Poverty Line'] = df_household['Total Household Count'] * df_poverty['All families']
    df_household['Total Single Generation Married Couples Under Poverty Line'] = df_household['Single Generation Married Couple'] * df_poverty['Married-couple family']
    try:
        df_household['Total Multi Generational Divorced Families Under Poverty Line'] = df_household['Multi Generational Divorced Family Households'] * df_poverty['Female householder, no spouse present, family']
    except:
        df_household['Total Multi Generational Divorced Families Under Poverty Line'] = df_household['Multi Generational Divorced Family Households'] * df_poverty['Female householder, no husband present, family']

    df_household= df_household[['Average Household Size', 'Average Family Size', 'Total Household Count', 'Multi Generational Family Households', 'Total Families Under Poverty Line', 'Single Generation Married Couple', 'Total Single Generation Married Couples Under Poverty Line', 'Multi Generational Divorced Family Households', 'Total Multi Generational Divorced Families Under Poverty Line', 'Non Family Households', 'Non Family Households (Living Alone)', 'Non Family Households (Not Living Alone)']]
    return df_household

In [67]:
df_poverty=poverty_rates('data/2022/POVERTY_RATES_FOR_FAMILIES_AND_PEOPLE_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED.csv',2022)
df_household=households('data/2022/HOUSEHOLDS_BY_TYPE.csv',2022)
vals=combine_frames(df_household, df_poverty)
vals

Label,Average Household Size,Average Family Size,Total Household Count,Multi Generational Family Households,Total Families Under Poverty Line,Single Generation Married Couple,Total Single Generation Married Couples Under Poverty Line,Multi Generational Divorced Family Households,Total Multi Generational Divorced Families Under Poverty Line,Non Family Households,Non Family Households (Living Alone),Non Family Households (Not Living Alone)
2022,2.69,3.07,32190,23112.42,1738.26,16545.66,496.3698,4957.26,500.68326,9077.58,6438.0,2671.77


In [72]:
def concat_data():
    years = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]
    df_list=[]
    for year in range(len(years)):
        try:
            household=households(f'data/{years[year]}/HOUSEHOLDS_BY_TYPE.CSV',years[year])
            poverty=poverty_rates(f'data/{years[year]}/POVERTY_RATES_FOR_FAMILIES_AND_PEOPLE_FOR_WHOM_POVERTY_STATUS_IS_DETERMINED.CSV',years[year])
            df_list.append(combine_frames(household,poverty))
        except:
            continue
    df= pd.concat(df_list)
    df.reset_index(inplace=True)
    df.rename(columns={'index':'Year'}, inplace=True)
    return df
    

In [73]:
HOUSEHOLDS_POVERTY_BY_TYPE = concat_data()
HOUSEHOLDS_POVERTY_BY_TYPE

Label,Year,Average Household Size,Average Family Size,Total Household Count,Multi Generational Family Households,Total Families Under Poverty Line,Single Generation Married Couple,Total Single Generation Married Couples Under Poverty Line,Multi Generational Divorced Family Households,Total Multi Generational Divorced Families Under Poverty Line,Non Family Households,Non Family Households (Living Alone),Non Family Households (Not Living Alone)
0,2010,2.81,3.40,24310,17211.480,753.610,13394.810,NaN,3233.230,NaN,7098.520,6028.880,1093.950
1,2011,3.09,3.82,22701,15096.165,862.638,11827.221,402.125514,2519.811,118.431117,7604.835,5788.755,1816.080
2,2012,2.97,3.48,23724,17176.176,1209.924,12407.652,297.783648,3819.564,515.641140,6547.824,4910.868,1613.232
3,2013,2.86,3.38,26959,18844.341,1455.786,13182.951,527.318040,3936.014,279.456994,8114.659,5957.939,2129.761
4,2014,3.17,3.82,22226,16002.720,822.362,12624.368,NaN,2756.024,NaN,6223.280,5178.658,1066.848
5,2015,2.85,3.48,25286,17523.198,758.580,13730.298,288.336258,2983.748,220.797352,7762.802,6144.498,1618.304
6,2016,2.97,3.52,23580,16317.360,1108.260,12143.700,412.885800,3419.100,297.461700,7262.640,5305.500,1933.560
7,2017,2.99,3.55,26139,18192.744,1516.062,12311.469,430.901415,4339.074,585.774990,7946.256,5855.136,2091.120
8,2018,2.78,3.33,28119,19120.920,506.142,14425.047,NaN,3289.923,NaN,8998.080,7226.583,1771.497
9,2019,2.93,3.54,26642,18303.054,506.198,14519.890,NaN,2371.138,NaN,8338.946,6394.080,1944.866


In [75]:
HOUSEHOLDS_POVERTY_BY_TYPE.to_csv('data/final/HOUSEHOLDS_BY_TYPE.CSV', index=False)